In this notebook, we prepare step by step the entraining plume. In short, we go through the following steps to compute the plume.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constants
cp = 1005.0
Rv = 461.5
Rd = 287.04
g = 9.81

In [ ]:
# Assume we have a profile of theta and q on height coordinates, and the surface pressure
L = 10_000
n = 256
p0 = 101300.0

dz = L / n
z = np.arange(dz/2, L, dz)
zh = np.arange(0, L+dz/2, dz)
theta = 300.0 + 0.003 * z
Hq = 2500
qt = 0.010 * np.exp(-z/Hq)

First, we assume that the environment is dry, and we compute the ambient pressure. We use the gas law for moist air.
$$
p = \rho R_\mathrm{d} \pi \theta_\mathrm{v}
$$
with $\pi = \left( \dfrac{p}{p_0} \right)^\dfrac{R_d}{c_p}$.

Then, we use hydrostatic balance and substitute the gas law:

$$
\begin{align}
\dfrac{\partial p}{\partial z}
&= -\rho g \\
&= -\dfrac{p g}{R_\mathrm{d} \pi \theta_\mathrm{v}} \\
\dfrac{\partial p^\frac{R_\mathrm{d}}{c_\mathrm{p}}}{\partial z}
&= -\dfrac{g}{c_\mathrm{p}}\dfrac{p_0^\frac{R_\mathrm{d}}{c_\mathrm{p}} }{\theta_\mathrm{v}} 
\end{align}
$$
with the latter equation, we can solve the pressure.

In [ ]:
# We assume the base state is dry
thetav = theta * (1 + (Rv/Rd - 1)*qt)

# Compute the pressure profile.
ph_Rdcp = np.zeros_like(zh)
ph_Rdcp[0] = p0**(Rd/cp)
for i in range(1, len(ph_Rdcp)):
    ph_Rdcp[i] = ph_Rdcp[i-1] - g/cp * p0**(Rd/cp) / thetav[i-1] * dz

ph = ph_Rdcp**(cp/Rd)
p = np.exp(0.5*(np.log(ph[:-1]) + np.log(ph[1:])))

In [ ]:
plt.figure(constrained_layout=True)
plt.subplot(131)
plt.plot(p, z)
plt.subplot(132)
plt.plot(theta, z)
plt.plot(thetav, z)
plt.subplot(133)
plt.plot(qt, z)

In [ ]:
# Define excess temperature and humidity (this can go into input).
dtheta = 1
dqv = 0.001